# 5.对于用户和菜肴打分的场景可构建如下图所示矩阵，
对该矩阵进行SVD 分解降维，在低维空间下利用item-based 协
同推荐方法进行推荐。（采用余弦相似性进行计算）

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import linalg as la

def euclidSim(inA, inB):
    return 1.0 / (1.0 + la.norm(inA - inB))

def pearsSim(inA,inB):
    if len(inA)<3:
        return 1.0
    return 0.5+0.5*np.corrcoef(inA,inB,rowvar=0)[0][1]

def cosSim(inA,inB):
    num=float((inA.T*inB)[0,0])
    denom=la.norm(inA)*la.norm(inB)
    return 0.5+0.5*(num/denom)

def standEst(dataMat,user,simMeas,item):
    n=np.shape(dataMat)[1]
    simTotal=0.0
    ratSimTotal=0.0
    for j in range(n):
        userRating=dataMat[user,j]
        if userRating==0:
            continue
        overLap=np.nonzero(np.logical_and(dataMat[:,item].A>0,dataMat[:,j].A>0))[0]
        if len(overLap)==0:
            similarity=0
        else:
            similarity=simMeas(dataMat[overLap,item],dataMat[overLap,j])
        print(f'the {item} and {j} similarity is: {similarity}')
        simTotal+=similarity
        ratSimTotal+=similarity*userRating
    if simTotal==0:
        return 0
    else:
        return ratSimTotal/simTotal

def recommend(dataMat,user,N=3,simMeas=cosSim,estMethod=standEst):
    unratedItems=np.nonzero(dataMat[user,:].A==0)[1]
    if len(unratedItems)==0:
        return 'you rated everything'
    itemScores=[]
    for item in unratedItems:
        estimatedScore=estMethod(dataMat,user,simMeas,item)
        itemScores.append((item,estimatedScore))
    return sorted(itemScores,key=lambda jj:jj[1],reverse=True)[:N]

def svdEst(dataMat,user,simMeas,item):
    n=np.shape(dataMat)[1]
    simTotal=0.0
    ratSimTotal=0.0
    U,Sigma,VT=la.svd(dataMat)
    Sig4=np.mat(np.eye(4)*Sigma[:4])
    xformedItems=dataMat.T*U[:,:4]*Sig4.I
    for j in range(n):
        userRating=dataMat[user,j]
        if userRating==0 or j ==item:
            continue
        similarity=simMeas(xformedItems[item,:].T,xformedItems[j,:].T)
        print(f'the {item} and {j} similarity is: {similarity}')
        simTotal+=similarity
        ratSimTotal+=similarity*userRating
    if simTotal==0:
        return 0
    else:
        return ratSimTotal/simTotal

data=np.mat([[2,0,0,4,4,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,5],[0,0,0,0,0,0,0,1,0,4,0],[3,3,4,0,3,0,0,2,2,0,0],[5,5,5,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,5,0,0,5,0],[4,0,4,0,0,0,0,0,0,0,5],[0,0,0,0,0,4,0,0,0,0,4],[0,0,0,0,0,0,5,0,0,5,0],[0,0,0,3,0,0,0,0,4,5,0],[1,1,2,1,1,2,1,0,4,5,0]])
print(data)

result=recommend(data,2,N=3,simMeas=cosSim,estMethod=standEst)
print(result)

[[2 0 0 4 4 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 5]
 [0 0 0 0 0 0 0 1 0 4 0]
 [3 3 4 0 3 0 0 2 2 0 0]
 [5 5 5 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 5 0 0 5 0]
 [4 0 4 0 0 0 0 0 0 0 5]
 [0 0 0 0 0 4 0 0 0 0 4]
 [0 0 0 0 0 0 5 0 0 5 0]
 [0 0 0 3 0 0 0 0 4 5 0]
 [1 1 2 1 1 2 1 0 4 5 0]]
the 0 and 7 similarity is: 1.0
the 0 and 9 similarity is: 1.0
the 1 and 7 similarity is: 1.0
the 1 and 9 similarity is: 1.0
the 2 and 7 similarity is: 1.0
the 2 and 9 similarity is: 1.0
the 3 and 7 similarity is: 0
the 3 and 9 similarity is: 0.9472135954999579
the 4 and 7 similarity is: 1.0
the 4 and 9 similarity is: 1.0
the 5 and 7 similarity is: 0
the 5 and 9 similarity is: 1.0
the 6 and 7 similarity is: 0
the 6 and 9 similarity is: 0.9446486458999437
the 8 and 7 similarity is: 1.0
the 8 and 9 similarity is: 0.9999999999999999
the 10 and 7 similarity is: 0
the 10 and 9 similarity is: 0
[(3, 4.0), (5, 4.0), (6, 4.0)]
